In [1]:
# import os
# import json
# import requests
# from datetime import datetime, timedelta

# class NasaClimateProvider:
#     def __init__(self):
#         self.evapotranspiration_calculator = EvapotranspirationCalculator()
#         self.timeout = 60
#         self.data_undefined = -999
#         self.expiry = timedelta(minutes=60)
#         self.start_date = datetime(1981, 1, 1)
#         self.end_date = datetime.now()

#     def is_cached(self, latitude, longitude):
#         path = self.get_cached_path(latitude, longitude)
#         return os.path.exists(path)

#     def get_custom_climate_data(self, latitude, longitude):
#         url = self.get_correct_api_url(latitude, longitude)
#         content = self.get_cached_data(latitude, longitude)
        
#         if not content:
#             print(f"Cached climate data for lat: {latitude}, long: {longitude} not found. Downloading now.")
#             try:
#                 response = requests.get(url, timeout=self.timeout)
#                 if response.status_code == 200:
#                     content = response.text
#                     print("NASA API content downloaded successfully.")
#                 else:
#                     raise Exception("NASA API couldn't be reached or connection timed out.")
#             except Exception as e:
#                 print(f"Could not load data from NASA API. Exception thrown: {e}")
#                 return []

#             if content:
#                 self.cache_data(latitude, longitude, content)
#             else:
#                 print("No content retrieved from API.")
#                 return []

#         data = self.process_data(content)
#         return data

#     def get_correct_api_url(self, latitude, longitude):
#         base_url = "https://power.larc.nasa.gov"
#         user_community = "AG"
#         parameters = "T2M,PRECTOTCORR,RH2M,ALLSKY_SFC_SW_DWN"
#         start_date_str = self.start_date.strftime("%Y%m%d")
#         end_date_str = self.end_date.strftime("%Y%m%d")
#         url = f"{base_url}/api/temporal/daily/point?parameters={parameters}&community={user_community}&longitude={longitude}&latitude={latitude}&start={start_date_str}&end={end_date_str}&format=JSON"
#         return url

#     def get_cached_data(self, latitude, longitude):
#         path = self.get_cached_path(latitude, longitude)
#         if os.path.exists(path):
#             file_info = os.stat(path)
#             last_access_time = datetime.fromtimestamp(file_info.st_atime)
#             if datetime.now() - last_access_time > self.expiry:
#                 return ""
#             else:
#                 os.utime(path, None)  # Update the access time
#                 with open(path, 'r') as file:
#                     return file.read()
#         return ""

#     def get_cached_path(self, latitude, longitude):
#         temp_dir = os.path.join(tempfile.gettempdir(), f"nasa_climate_data_lat_{latitude}_long_{longitude}")
#         return temp_dir

#     def cache_data(self, latitude, longitude, content):
#         path = self.get_cached_path(latitude, longitude)
#         with open(path, 'w') as file:
#             file.write(content)

#     def process_data(self, content):
#         data = json.loads(content)
#         properties = data.get('properties', {})
#         daily_data = []

#         for param, values in properties.items():
#             for date_str, value in values.items():
#                 # Example of processing the data
#                 data_obj = {
#                     'date': datetime.strptime(date_str, "%Y%m%d"),
#                     'value': value
#                 }
#                 daily_data.append(data_obj)

#         return daily_data

# # This class needs to be implemented according to the specific methods used for evapotranspiration calculation.
# class EvapotranspirationCalculator:
#     def calculate_reference_evapotranspiration(self, temperature, solar_radiation, humidity):
#         # Placeholder for actual calculation
#         return temperature + solar_radiation + humidity



In [1]:
import requests
import json
from datetime import datetime

def fetch_growing_season_data(latitude, longitude, year):
    """
    Fetch growing season data from NASA POWER API for specified parameters.

    Parameters:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        year (int): Year for which data is requested.

    Returns:
        dict: Dictionary containing growing season total precipitation and other parameters.
    """
    start_date = f"{year}0501"  # May 1st
    end_date = f"{year}0930"    # September 30th

    # Define the parameters and the community
    parameters = "PRECTOTCORR,T2M,RH2M,ALLSKY_SFC_SW_DWN"
    community = "AG"

    # Construct the API request URL
    base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    api_request_url = (
        f"{base_url}?parameters={parameters}&community={community}"
        f"&longitude={longitude}&latitude={latitude}&start={start_date}&end={end_date}&format=JSON"
    )

    # Make the API request
    response = requests.get(api_request_url, timeout=30.00)
    if response.status_code != 200:
        raise Exception("Failed to fetch data from NASA POWER API")

    # Load JSON response content
    data = json.loads(response.content.decode('utf-8'))
    
    # Initialize variables to store total precipitation and other data
    total_precipitation = 0
    daily_temperature = []
    relative_humidity = []
    solar_radiation = []

    # Extract data for each day in the response
    for day in data['properties']['parameter']['PRECTOTCORR']:
        total_precipitation += data['properties']['parameter']['PRECTOTCORR'][day]
        daily_temperature.append(data['properties']['parameter']['T2M'][day])
        relative_humidity.append(data['properties']['parameter']['RH2M'][day])
        solar_radiation.append(data['properties']['parameter']['ALLSKY_SFC_SW_DWN'][day])

    # Organize results into a dictionary
    result = {
        'Total Precipitation (mm)': total_precipitation,
        'Average Daily Temperature (C)': sum(daily_temperature) / len(daily_temperature),
        'Average Relative Humidity (%)': sum(relative_humidity) / len(relative_humidity),
        'Average Solar Radiation (MJ m^-2 day^-1)': sum(solar_radiation) / len(solar_radiation)
    }

    return result


In [2]:
# Example usage
latitude = 46.476
longitude = -71.519 
year = 2021

try:
    growing_season_data = fetch_growing_season_data(latitude, longitude, year)
    print(growing_season_data)
except Exception as e:
    print(f"Error: {e}")

{'Total Precipitation (mm)': 403.52999999999975, 'Average Daily Temperature (C)': 16.634640522875824, 'Average Relative Humidity (%)': 81.08287581699346, 'Average Solar Radiation (MJ m^-2 day^-1)': 18.19588235294118}


In [5]:
import requests
import json
from datetime import datetime
from evapotranspiration_calculator import EvapotranspirationCalculator

def fetch_growing_season_data(latitude, longitude, year):
    """
    Fetch growing season data from NASA POWER API and calculates total evapotranspiration.

    Parameters:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        year (int): Year for which data is requested.

    Returns:
        dict: Dictionary containing total precipitation, total evapotranspiration, and other parameters.
    """
    start_date = f"{year}0501"  # May 1st
    end_date = f"{year}0930"    # September 30th

    # Define the parameters and the community
    parameters = "PRECTOTCORR,T2M,RH2M,ALLSKY_SFC_SW_DWN"
    community = "AG"

    # Construct the API request URL
    base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    api_request_url = (
        f"{base_url}?parameters={parameters}&community={community}"
        f"&longitude={longitude}&latitude={latitude}&start={start_date}&end={end_date}&format=JSON"
    )

    # Make the API request
    response = requests.get(api_request_url, timeout=30.00)
    if response.status_code != 200:
        raise Exception("Failed to fetch data from NASA POWER API")

    # Load JSON response content
    data = json.loads(response.content.decode('utf-8'))
    
    # Initialize variables to store total precipitation and other data
    total_precipitation = 0
    total_evapotranspiration = 0

    # Extract data for each day in the response
    for day in data['properties']['parameter']['PRECTOTCORR']:
        total_precipitation += data['properties']['parameter']['PRECTOTCORR'][day]
        
        # Initialize EvapotranspirationCalculator with daily values
        calculator = EvapotranspirationCalculator(
            mean_daily_temperature=data['properties']['parameter']['T2M'][day],
            solar_radiation=data['properties']['parameter']['ALLSKY_SFC_SW_DWN'][day],
            relative_humidity=data['properties']['parameter']['RH2M'][day]
        )
        daily_evapotranspiration = calculator.calculate()
        total_evapotranspiration += daily_evapotranspiration

    # Organize results into a dictionary
    result = {
        'Total Precipitation (mm)': total_precipitation,
        'Total Evapotranspiration (mm)': total_evapotranspiration
    }

    return result



In [6]:
# Example usage
latitude = 46.476
longitude = -71.519 
year = 2021

try:
    growing_season_data = fetch_growing_season_data(latitude, longitude, year)
    print(growing_season_data)
except Exception as e:
    print(f"Error: {e}")


{'Total Precipitation (mm)': 403.52999999999975, 'Total Evapotranspiration (mm)': 493.59461920343324}
